In [306]:
from pymongo import MongoClient
import pandas as pd
import re
import folium

In [307]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [308]:
db, coll = connectCollection('companies','companies')
print(coll.json)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies.json')


In [309]:
extraer = [
    { "$unwind": "$offices"},
    {"$match": {"$and":
            [{"founded_year": {"$gte": 2009}},
             {"deadpooled_year": None},
             {"total_money_raised": {"$exists": True, "$ne": "$0"}},
             {"offices.latitude": {"$exists": True, "$ne": None}},
             {"offices.longitude": {"$exists": True, "$ne": None}}]}}]

oficinas_fuera = list(coll.aggregate(extraer))


print(oficinas_fuera[0])

{'_id': ObjectId('52cdef7c4bab8bd675297f94'), 'name': 'PeekYou', 'permalink': 'peekyou', 'crunchbase_url': 'http://www.crunchbase.com/company/peekyou', 'homepage_url': 'http://www.peekyou.com', 'blog_url': 'http://www.peekyou.biz', 'blog_feed_url': 'http://blog.peekyou.com/feed/', 'twitter_username': 'peekyou', 'category_code': 'search', 'number_of_employees': 20, 'founded_year': 2012, 'founded_month': 11, 'founded_day': 2, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': 'social-crm, database, peekyou, peoplesearch, engine, people, search, directory, social, google', 'alias_list': None, 'email_address': 'info@peekyou.com', 'phone_number': '', 'description': '', 'created_at': 'Fri Aug 31 12:46:53 UTC 2007', 'updated_at': 'Wed Jul 10 10:13:16 UTC 2013', 'overview': '<p>PeekYou places the individual at the center of the Internet. For every public weblink, PeekYou tries to answer the question, &#8220;Who made it?&#8221; or, whe

In [310]:
def getLocation(companies):
    for i in range(len(companies['offices'])):
        longitude = companies['offices']['longitude']
        latitude = companies['offices']['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        return loc

In [311]:
getLocation(oficinas_fuera[1])

{'type': 'Point', 'coordinates': [-111.9035, 33.8171]}

In [312]:
for cp in oficinas_fuera:
    value = {"$set": {'geolocation':getLocation(cp)}}
    coll.update_one(cp, value)

In [313]:
print(oficinas_fuera[0])

{'_id': ObjectId('52cdef7c4bab8bd675297f94'), 'name': 'PeekYou', 'permalink': 'peekyou', 'crunchbase_url': 'http://www.crunchbase.com/company/peekyou', 'homepage_url': 'http://www.peekyou.com', 'blog_url': 'http://www.peekyou.biz', 'blog_feed_url': 'http://blog.peekyou.com/feed/', 'twitter_username': 'peekyou', 'category_code': 'search', 'number_of_employees': 20, 'founded_year': 2012, 'founded_month': 11, 'founded_day': 2, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': 'social-crm, database, peekyou, peoplesearch, engine, people, search, directory, social, google', 'alias_list': None, 'email_address': 'info@peekyou.com', 'phone_number': '', 'description': '', 'created_at': 'Fri Aug 31 12:46:53 UTC 2007', 'updated_at': 'Wed Jul 10 10:13:16 UTC 2013', 'overview': '<p>PeekYou places the individual at the center of the Internet. For every public weblink, PeekYou tries to answer the question, &#8220;Who made it?&#8221; or, whe

In [314]:
companies_df = pd.DataFrame(oficinas_fuera)

companies_df


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners,location,geolocation
0,52cdef7c4bab8bd675297f94,PeekYou,peekyou,http://www.crunchbase.com/company/peekyou,http://www.peekyou.com,http://www.peekyou.biz,http://blog.peekyou.com/feed/,peekyou,search,20.0,...,[],"{'description': None, 'address1': None, 'addre...","[{'id': 18106, 'description': 'Launched in Ste...",None,"[{'embed_code': '<object width=""420"" height=""2...",[],[{'external_url': 'http://www.webmetricsguru.c...,[],"[{'type': 'Point', 'coordinates': [-73.985506,...","{'type': 'Point', 'coordinates': [-73.985506, ..."
1,52cdef7c4bab8bd675297f9e,GENWI,genwi,http://www.crunchbase.com/company/genwi,http://www.genwi.com,http://blog.genwi.com,,genwi,mobile,25.0,...,[],"{'description': '', 'address1': '4966 El Camin...","[{'id': 14542, 'description': 'Rahul Patel, fo...",None,[],"[{'available_sizes': [[[150, 96], 'assets/imag...","[{'external_url': 'http://iSites.us', 'title':...",[],"[{'type': 'Point', 'coordinates': [-111.9035, ...","{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,52cdef7c4bab8bd675297fec,Fixya,fixya,http://www.crunchbase.com/company/fixya,http://www.fixya.com,http://blog.muktar.com,http://blog.muktar.com/feed/,MUaliyu,web,30.0,...,[],"{'description': '', 'address1': 'One Franklin ...",[],None,"[{'embed_code': '<div style=""width:320px;""><di...",[],[{'external_url': 'http://www.vccafe.com/2009/...,[],"[{'type': 'Point', 'coordinates': [-122.323895...","{'type': 'Point', 'coordinates': [-122.323895,..."
3,52cdef7c4bab8bd6752981d8,Widgetbox,widgetbox,http://www.crunchbase.com/company/widgetbox,http://widgetbox.com,,http://blog.widgetbox.com/feed/,widgetbox,other,30.0,...,[],"{'description': None, 'address1': None, 'addre...","[{'id': 1669, 'description': 'Widgetizing The ...",None,[],"[{'available_sizes': [[[100, 75], 'assets/imag...",[],[],"[{'type': 'Point', 'coordinates': [-122.398599...","{'type': 'Point', 'coordinates': [-122.398599,..."
4,52cdef7c4bab8bd6752981dc,RazorGator,razorgator,http://www.crunchbase.com/company/razorgator,http://razorgator.com,,,Razorgator,ecommerce,NaN,...,[],"{'description': None, 'address1': '11150 Santa...",[],None,[],[],[],[],"[{'type': 'Point', 'coordinates': [-118.445243...","{'type': 'Point', 'coordinates': [-118.445243,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,52cdef7f4bab8bd67529c2d0,Delphi,delphi,http://www.crunchbase.com/company/delphi,http://www.delphi.com,,,DelphiAuto,cleantech,NaN,...,[],"{'description': '', 'address1': '5725 Delphi D...",[],None,[],"[{'available_sizes': [[[150, 75], 'assets/imag...",[],[],"[{'type': 'Point', 'coordinates': [-83.16134, ...","{'type': 'Point', 'coordinates': [-83.16134, 4..."
102,52cdef7f4bab8bd67529c343,ChallengePost,challengepost,http://www.crunchbase.com/company/challengepost,http://www.challengepost.com,http://blog.challengepost.com,,challengepost,web,17.0,...,[],"{'description': '', 'address1': '425 West 13th...",[],None,[],"[{'available_sizes': [[[150, 136], 'assets/ima...",[],[],"[{'type': 'Point', 'coordinates': [-74.00717, ...","{'type': 'Point', 'coordinates': [-74.00717, 4..."
103,52cdef7f4bab8bd67529c448,Teambox,teambox,http://www.crunchbase.com/company/teambox,http://www.teambox.com,http://teambox.com/blog,,teambox_app,web,35.0,...,"[{'price_amount': None, 'price_currency_code':...","{'description': 'Barcelona Office', 'address1'...","[{'id': 23211, 'description': 'Teambox hires D...",None,"[{'embed_code': '<iframe src="""">Teambox online...","[{'available_sizes': [[[150, 83], 'assets/imag...",[],"[{'partner_name': 'AngelList', 'homepage_url':...",NaN,"{'type': 'Point', 'coordinates': [2.1920231, 4..."
104,52cdef7f4bab8bd67529c6b6,Tiny Speck,tiny-speck,http://www.crunchbase.com/company/tiny-speck,http://tinyspeck.com,,,slackhq,games_video,45.0,...,[],

In [315]:
print(len(companies_df))


106


In [316]:
companies_df1 = companies_df[["name","category_code","founded_year","total_money_raised","geolocation"]]

companies_df1
    


,name,category_code,founded_year,total_money_raised,geolocation
0,PeekYou,search,2012,$1.83M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,GENWI,mobile,2010,$7.1M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,Fixya,web,2013,$8M,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,Widgetbox,other,2012,$14.5M,"{'type': 'Point', 'coordinates': [-122.398599,..."
4,RazorGator,ecommerce,2011,$58.8M,"{'type': 'Point', 'coordinates': [-118.445243,..."
...,...,...,...,...,...
101,Delphi,cleantech,2009,$8M,"{'type': 'Point', 'coordinates': [-83.16134, 4..."
102,ChallengePost,web,2009,$4.6M,"{'type': 'Point', 'coordinates': [-74.00717, 4..."
103,Teambox,web,2009,$6.75M,"{'type': 'Point', 'coordinates': [2.1920231, 4..."
104,Tiny Speck,games_video,2009,$17.2M,"{'type': 'Point', 'coordinates': [-123.109217,..."


In [317]:
companies_df1['category_code'].value_counts()

web                15
games_video        14
software           14
advertising        14
enterprise          9
mobile              8
travel              5
ecommerce           5
biotech             4
social              2
search              2
cleantech           2
other               2
education           2
hardware            2
medical             1
fashion             1
network_hosting     1
security            1
analytics           1
messaging           1
Name: category_code, dtype: int64

In [318]:
category_list = {'web': 'tech', 'games_video': 'tech', 'software':'tech', 'advertising':'tech', 
                 'mobile':'tech', 'ecommerce':'tech', 'biotech':'tech', 'search':'tech',
                 'cleantech':'tech','hardware':'tech', 'netword_hosting':'tech', 'security':'tech', 
                 'analytics':'tech'}

In [319]:
companies_df2 = companies_df1.replace(to_replace = category_list, inplace=False) 

companies_df2.head(40)


,name,category_code,founded_year,total_money_raised,geolocation
0,PeekYou,tech,2012,$1.83M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,GENWI,tech,2010,$7.1M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,Fixya,tech,2013,$8M,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,Widgetbox,other,2012,$14.5M,"{'type': 'Point', 'coordinates': [-122.398599,..."
4,RazorGator,tech,2011,$58.8M,"{'type': 'Point', 'coordinates': [-118.445243,..."
5,Social Gaming Network,tech,2011,$17.1M,"{'type': 'Point', 'coordinates': [-122.161523,..."
6,Social Gaming Network,tech,2011,$17.1M,"{'type': 'Point', 'coordinates': [-122.161523,..."
7,Skydeck,tech,2012,$4M,"{'type': 'Point', 'coordinates': [-122.3254705..."
8,Handipoints,tech,2009,$800k,"{'type': 'Point', 'coordinates': [-122.214897,..."
9,Magento,tech,2010,$22.5M,"{'type': 'Point', 'coordinates': [-118.243425,..."


In [320]:
companies_df3 = companies_df2[['tech' in x for x in companies_df2['category_code']]]
companies_df3


,name,category_code,founded_year,total_money_raised,geolocation
0,PeekYou,tech,2012,$1.83M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,GENWI,tech,2010,$7.1M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,Fixya,tech,2013,$8M,"{'type': 'Point', 'coordinates': [-122.323895,..."
4,RazorGator,tech,2011,$58.8M,"{'type': 'Point', 'coordinates': [-118.445243,..."
5,Social Gaming Network,tech,2011,$17.1M,"{'type': 'Point', 'coordinates': [-122.161523,..."
...,...,...,...,...,...
101,Delphi,tech,2009,$8M,"{'type': 'Point', 'coordinates': [-83.16134, 4..."
102,ChallengePost,tech,2009,$4.6M,"{'type': 'Point', 'coordinates': [-74.00717, 4..."
103,Teambox,tech,2009,$6.75M,"{'type': 'Point', 'coordinates': [2.1920231, 4..."
104,Tiny Speck,tech,2009,$17.2M,"{'type': 'Point', 'coordinates': [-123.109217,..."


In [321]:
companies_df3.total_money_raised = companies_df3.total_money_raised.str.replace(r'(\d+)([A-Za-z])', r'\1 \2')

companies_df3


,name,category_code,founded_year,total_money_raised,geolocation
0,PeekYou,tech,2012,$1.83 M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,GENWI,tech,2010,$7.1 M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,Fixya,tech,2013,$8 M,"{'type': 'Point', 'coordinates': [-122.323895,..."
4,RazorGator,tech,2011,$58.8 M,"{'type': 'Point', 'coordinates': [-118.445243,..."
5,Social Gaming Network,tech,2011,$17.1 M,"{'type': 'Point', 'coordinates': [-122.161523,..."
...,...,...,...,...,...
101,Delphi,tech,2009,$8 M,"{'type': 'Point', 'coordinates': [-83.16134, 4..."
102,ChallengePost,tech,2009,$4.6 M,"{'type': 'Point', 'coordinates': [-74.00717, 4..."
103,Teambox,tech,2009,$6.75 M,"{'type': 'Point', 'coordinates': [2.1920231, 4..."
104,Tiny Speck,tech,2009,$17.2 M,"{'type': 'Point', 'coordinates': [-123.109217,..."


In [322]:
companies_df3 = companies_df3[['M' in x for x in companies_df3['total_money_raised']]]
companies_df3


,name,category_code,founded_year,total_money_raised,geolocation
0,PeekYou,tech,2012,$1.83 M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,GENWI,tech,2010,$7.1 M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,Fixya,tech,2013,$8 M,"{'type': 'Point', 'coordinates': [-122.323895,..."
4,RazorGator,tech,2011,$58.8 M,"{'type': 'Point', 'coordinates': [-118.445243,..."
5,Social Gaming Network,tech,2011,$17.1 M,"{'type': 'Point', 'coordinates': [-122.161523,..."
...,...,...,...,...,...
101,Delphi,tech,2009,$8 M,"{'type': 'Point', 'coordinates': [-83.16134, 4..."
102,ChallengePost,tech,2009,$4.6 M,"{'type': 'Point', 'coordinates': [-74.00717, 4..."
103,Teambox,tech,2009,$6.75 M,"{'type': 'Point', 'coordinates': [2.1920231, 4..."
104,Tiny Speck,tech,2009,$17.2 M,"{'type': 'Point', 'coordinates': [-123.109217,..."


In [323]:
print(companies_df3.reset_index(drop=True))

                     name category_code  founded_year total_money_raised  \
0                 PeekYou          tech          2012            $1.83 M   
1                   GENWI          tech          2010             $7.1 M   
2                   Fixya          tech          2013               $8 M   
3              RazorGator          tech          2011            $58.8 M   
4   Social Gaming Network          tech          2011            $17.1 M   
..                    ...           ...           ...                ...   
56                 Delphi          tech          2009               $8 M   
57          ChallengePost          tech          2009             $4.6 M   
58                Teambox          tech          2009            $6.75 M   
59             Tiny Speck          tech          2009            $17.2 M   
60                 Getyoo          tech          2009            €1.13 M   

                                          geolocation  
0   {'type': 'Point', 'coordina

In [324]:

companies_df3.head(20)

,name,category_code,founded_year,total_money_raised,geolocation
0,PeekYou,tech,2012,$1.83 M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,GENWI,tech,2010,$7.1 M,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,Fixya,tech,2013,$8 M,"{'type': 'Point', 'coordinates': [-122.323895,..."
4,RazorGator,tech,2011,$58.8 M,"{'type': 'Point', 'coordinates': [-118.445243,..."
5,Social Gaming Network,tech,2011,$17.1 M,"{'type': 'Point', 'coordinates': [-122.161523,..."
...,...,...,...,...,...
101,Delphi,tech,2009,$8 M,"{'type': 'Point', 'coordinates': [-83.16134, 4..."
102,ChallengePost,tech,2009,$4.6 M,"{'type': 'Point', 'coordinates': [-74.00717, 4..."
103,Teambox,tech,2009,$6.75 M,"{'type': 'Point', 'coordinates': [2.1920231, 4..."
104,Tiny Speck,tech,2009,$17.2 M,"{'type': 'Point', 'coordinates': [-123.109217,..."


In [325]:
companies_df3.to_json('./mycompanies_clean.json', orient='records')